In [70]:
import pandas as pd

from Project.Database import Db
from optimisation_problem import hourly_house_df, slice_emission_vector, power_consumption_vector, NZERTF_optimiser, SE_time_df, json_to_dataframe

In [71]:
def find_emissions(df, emission_vec):
    energy_vec = df.copy()['Consumption'].reset_index(drop=True)
    emission_vector = energy_vec.multiply(emission_vec)
    for index, emission in emission_vector[lambda self: self > 0].iteritems():
        df.loc[lambda self: (self['Hour'] == index), 'Emission'] += emission
    return df['Emission']

In [72]:
meta = Db.load_data(meta=True, hourly=False, year=2, consumption=False).loc[
    lambda self: (~self['Consumer_Match'].isna()), 'Consumer_Match']

movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher',
                      'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

appliances = meta.index.tolist()
consumers = meta.tolist()

patterns = SE_time_df(
    dataframe=json_to_dataframe(
        year=2,
        level=1,
        exclude_follows=True,
        with_redundancy=False).loc[lambda self: self['pattern'].isin(movable_appliances)],
    TAT=0.1,
    TS_method='mean')

power_consumption = power_consumption_vector(movable_appliances=movable_appliances)

# with redundancy <- w.r
# without redundancy <- w.o.r
# movable appliances unoptimised <- m.a.u.o
# movable appliances optimised <- m.a.o
NZERTF_optimisation = {
    'w.r': Db.load_data(year=2, hourly=False),
    'w.o.r': Db.load_data(year=2, hourly=False, with_redundancy=False)
}

NZERTF_optimisation.update({
    'm.a.u.o': NZERTF_optimisation['w.o.r'].copy()[['Timestamp'] + movable_appliances]
})

timestamps = pd.to_datetime(
    NZERTF_optimisation['w.r']['Timestamp'].dt.strftime('%Y-%m-%d %H').unique(),
    format='%Y-%m-%d %H')

production = Db.load_data(
    consumption=False,
    production=True,
    year=2)['CO2(Grams)/kWh'][lambda self: self.index.isin(timestamps)]
production = production.groupby(pd.to_datetime(production.index.strftime('%Y-%m-%d %H'))).sum()

for key, value in NZERTF_optimisation.items():
    df = value.copy()
    if all([att in df.columns for att in consumers]):
        df =  hourly_house_df(house_df=df.copy(), aggregate_func='mean')
        df['Consumption'] = df[consumers].div(1_000).sum(1)
        df.drop(labels=consumers, inplace=True, axis=1)
    else:
        df = hourly_house_df(house_df=df, aggregate_func='mean')

        df['Consumption'] = df[movable_appliances].dot(
            power_consumption).div(1_000)

    for day in df['Day'].unique():
        df.loc[(lambda self: (self['Day'] == day)),
               'Emission'] = find_emissions(df=df.loc[lambda self: self['Day'] == day],
                                            emission_vec=slice_emission_vector(
                                                production_vectors=production,
                                                day_number=day))
    NZERTF_optimisation.update({
        key: df
    })

NZERTF_optimisation.update({
    'm.a.o': NZERTF_optimiser()
})

NZERTF_emission = {}

for key in NZERTF_optimisation.keys():
    NZERTF_emission.update({
        key: NZERTF_optimisation[key]['Emission'].sum()
    })

/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_5862/1747781345.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[lambda self: (self['Hour'] == index), 'Emission'] += emission
/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_5862/1747781345.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[lambda self: (self['Hour'] == index), 'Emission'] += emission
/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_5862/1747781345.py:5: SettingWithCopyWarning: 
A value is try

In [73]:
for key, value in NZERTF_optimisation.items():
    print(f'{key}: {value["Emission"]}')

w.r: 0        508.534908
1        709.815540
2        523.886634
3        508.910148
4        522.700297
           ...     
8667    1848.906611
8668    1503.742106
8669    1989.126381
8670    1720.940530
8671    1102.978491
Name: Emission, Length: 8672, dtype: float64
w.o.r: 0        508.534908
1        709.815540
2        523.886634
3        508.910148
4        522.700297
           ...     
8667    1646.223306
8668    1412.316982
8669    1989.126381
8670    1697.332044
8671    1099.954859
Name: Emission, Length: 8672, dtype: float64
m.a.u.o: 0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
           ...    
8667      0.000000
8668    119.485024
8669    217.245498
8670    144.830332
8671      0.000000
Name: Emission, Length: 8672, dtype: float64
m.a.o: 0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.000000
           ...     
8667       0.000000
8668      48.797821
8669    1069.285258
8670    

In [74]:
NZERTF_emission

{'w.r': 8371902.086841607,
 'w.o.r': 8242621.19712289,
 'm.a.u.o': 511242.76383277134,
 'm.a.o': 539569.892249094}

In [75]:
redundancy_reduction = NZERTF_emission['w.r'] - NZERTF_emission['w.o.r']
optimisation_reduction = NZERTF_emission['m.a.u.o'] - NZERTF_emission['m.a.o']
total_reduction = redundancy_reduction + optimisation_reduction

In [76]:
redundancy_reduction

129280.88971871696

In [77]:
optimisation_reduction

-28327.128416322637

In [78]:
total_reduction

100953.76130239433

In [79]:
NZERTF_optimisation['m.a.o']

,Day,Hour,Timestamp,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission,CumulativeEmission
0,0,0,2015-02-01 00:00:00,0,0,0,0,0,0.000000,0.000000
1,0,1,2015-02-01 01:00:00,0,0,0,0,0,0.000000,0.000000
2,0,2,2015-02-01 02:00:00,0,0,0,0,0,0.000000,0.000000
3,0,3,2015-02-01 03:00:00,0,0,0,0,0,0.000000,0.000000
4,0,4,2015-02-01 04:00:00,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
8667,364,19,2016-01-31 19:00:00,0,0,0,0,0,0.000000,538303.572346
8668,364,20,2016-01-31 20:00:00,0,0,1,0,0,48.797821,538352.370166
8669,364,21,2016-01-31 21:00:00,1,0,1,0,0,1069.285258,539421.655424
8670,364,22,2016-01-31 22:00:00,1,0,0,0,0,118.589460,539540.244884


In [80]:
NZERTF_optimisation['m.a.u.o']

,Day,Hour,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission,Consumption
0,0,0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
1,0,1,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
2,0,2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
3,0,3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
4,0,4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...
8667,364,19,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
8668,364,20,0.550000,0.0,0.0,0.0,0.0,119.485024,0.1551
8669,364,21,1.000000,0.0,0.0,0.0,0.0,217.245498,0.2820
8670,364,22,0.666667,0.0,0.0,0.0,0.0,144.830332,0.1880


In [81]:
NZERTF_optimisation['w.r']['Consumption'].sum()-NZERTF_optimisation['w.o.r']['Consumption'].sum()

182.9747460944709

In [82]:
NZERTF_optimisation['w.r']['Consumption'].groupby(NZERTF_optimisation['w.r']['Consumption'].index % 24).mean()

0     0.725440
1     0.718262
2     0.773034
3     0.806631
4     0.782573
5     2.778843
6     1.744991
7     1.632203
8     1.417562
9     1.147923
10    0.625064
11    1.188962
12    0.759670
13    0.988327
14    1.124459
15    0.726056
16    0.761961
17    2.725864
18    2.552129
19    2.482216
20    2.286384
21    2.033767
22    1.238514
23    0.772881
Name: Consumption, dtype: float64